In [1]:
import pandas as pd
import numpy as np


In [2]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer

In [3]:
from sklearn.tree import DecisionTreeClassifier

In [4]:
from preprocessing import traer_datasets
from preprocessing import separar_dataset
from preprocessing import feature_engineering_general

Traemos datasets y particionamos

In [5]:
df, df_sin_target, solo_target = traer_datasets()
X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)
X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_train.is_copy=False

Aplicamos metodo de feature engineering general

In [6]:
X_train, X_test = feature_engineering_general(X_train, X_test)
X_train.head(20)


,horas_de_sol,humedad_tarde,humedad_temprano,mm_evaporados_agua,mm_lluvia_dia,nubosidad_tarde,nubosidad_temprano,presion_atmosferica_tarde,presion_atmosferica_temprano,rafaga_viento_max_velocidad,temp_max,temp_min,temperatura_tarde,temperatura_temprano,velocidad_viendo_tarde,velocidad_viendo_temprano,llovieron_hamburguesas_hoy_si
id,,,,,,,,,,,,,,,,,
0.0,7.160884,22.0,71.0,6.699027,0.600000,5.010961,8.000000,1007.1,1007.7,44.0,22.9,13.4,21.8,16.9,24.0,20.000000,0.0
1.0,9.887203,25.0,44.0,6.456726,0.000000,3.708656,2.916872,1007.8,1010.6,44.0,25.1,7.4,24.3,17.2,22.0,4.000000,0.0
2.0,11.558929,30.0,38.0,8.706878,0.000000,2.000000,2.556847,1008.7,1007.6,46.0,25.7,12.9,23.2,21.0,26.0,19.000000,0.0
3.0,10.316795,16.0,45.0,6.865982,0.000000,3.256263,2.444697,1012.8,1017.6,24.0,28.0,9.2,26.5,18.1,9.0,11.000000,0.0
4.0,5.593569,33.0,82.0,7.748400,1.000000,8.000000,7.000000,1006.0,1010.8,41.0,32.3,17.5,29.7,17.8,20.0,7.000000,0.0
5.0,9.529683,23.0,55.0,8.215936,0.200000,3.862529,3.759482,1005.4,1009.2,56.0,29.7,14.6,28.9,20.6,24.0,19.000000,0.0
6.0,11.050696,19.0,49.0,8.050543,0.000000,2.608826,1.000000,1008.2,1009.6,50.0,25.0,14.3,24.6,18.1,24.0,20.000000,0.0
7.0,10.242604,19.0,48.0,6.723142,0.000000,3.454650,2.871526,1010.1,1013.4,35.0,26.7,7.7,25.5,16.3,17.0,6.000000,0.0
8.0,10.480564,9.0,42.0,9.149061,0.000000,3.634536,2.656091,1003.6,1008.9,80.0,31.9,9.7,30.2,18.3,28.0,7.000000,0.0


array([0., 1.])